## Setting up an AWS server

### Introduction to storage

EBS allows us to define one or more storage volumes of almost any size we wish. The hardware details of that storage will be handled for us behind the scenes. EBS volumes exist independently of an EC2 machine and can be moved from one machine to another and the contents can be stored and attached to multiple EC2 instances over time (though only one instance at a time).

An AMI or Amazon Machine Instance is a template for launching an instance. The AMI includes a template for a pre-configured root volume that contains an operating system (e.g. Ubuntu Linux, Windows, etc.). The AMI also includes basic configuration of storage volumes that will be available within an instance. You can store a configured machine as an AMI to save your work. There are community AMIs that can be shared.

### SSH into the server

cd AWS-Tutorial  
chmod 400 MyFirstKey.pem   
ssh -i /Users/cameronstrachan/Desktop/scripts/MyFirstKey.pem -o ServerAliveInterval=60 ubuntu@52.40.146.10

### Connect storage volumes to server and set up file structure

sudo mkfs -t ext4 /dev/xvdc  
sudo mkdir data  
sudo mount /dev/xvdc data  
sudo chmod 777 data  
cd data  
mkdir output  
mkdir tools 

## Setting up on Azure

Storage seems to be already attached to the server.

### Making the ssh key  and SSH into server
ssh-keygen -t rsa -b 2048 -C "<your_user@yourdomain.com>"  
ssh-add /Users/cameronstrachan/.ssh/id_rsa  
  
Use this to copy the key so that it can be pasted into the azure sever set up page:
pbcopy < ~/.ssh/id_rsa.pub  
  
When using ssh, note the uder name is definted from the azure set up:  
ssh -i /Users/cameronstrachan/.ssh/id_rsa -o ServerAliveInterval=60 cam@40.121.90.130  
  
A final precaution for preventing the broken pipe error, type the following command into a local terminal:  
echo "ClientAliveInterval 60" | sudo tee -a /etc/ssh/sshd_config  

## Running SPAdes

### Downloading SPAdes and upload data

cd tools  
wget http://spades.bioinf.spbau.ru/release3.7.1/SPAdes-3.7.1-Linux.tar.gz  
tar -xzf SPAdes-3.7.1-Linux.tar.gz  
cd SPAdes-3.7.1-Linux/bin/  
  
Upload the data to the server by running scp from local machine (open new terminal window). 
  
scp -r -i /Users/cameronstrachan/Desktop/scripts/MyFirstKey.pem /Users/cameronstrachan/Desktop/upload/ ubuntu@52.40.146.10:/home/ubuntu/data  
  
Once the data is uploaded, unzip the files on the server.   
    
cd input  
gunzip *.gz  

### Making command strings  

On local machine run the following python script. 

In [ ]:
import os
import errno

try:
    os.makedirs("./input/commands/")
except OSError as exc: 
    if exc.errno == errno.EEXIST and os.path.isdir("./input/commands/"):
        pass
    else:
        raise

filelist = [f for f in os.listdir("./input/") if f.endswith(".gz")]
names = []


for k in range (0,len(filelist)):
    files1=filelist[k]
    files2=files1.split("_")
    names.append(files2[0])

names = sorted(set(names))
names = list(names)


for j in names:
    outfile = open("./input/commands/"+str(j),"w")
    command = []
    for l in range (0,len(filelist)):
        files3=filelist[l]
        files4=files3.split("_")
        if j == files4[0]:
            library = files4[2]
            librarynum = list(library)
            paired = files4[3]
            pairednum = list(paired)
            command.append("--pe" + str(librarynum[3]) + "-" + str(pairednum[1]) + " input/" + filelist[l] + " ")
        else:
            pass
    outfile.write("python tools/SPAdes-3.7.1-Linux/bin/spades.py " + " ".join(command) + "-o output/" + str(j) + "/" + "\n")
    outfile.close()


outfile = open("./input/commands/mkdirectory","w")
for j in names:
    outfile.write("mkdir output/" + str(j) + "\n")
outfile.close()

After running the above script, run the following commands in terminal:  
  
python make_commands2.py  
cd input/commands  
cat mkdirectory *x > commands.txt  
sed -i '' 's/.gz//g' commands.txt  
  
The commands are ready to copy and run on the server. 

### Running SPAdes on the server
  
1. First, use the mkdir commands to make directories in the output folder. This is where the SPAdes data will be stored.  
2. Open a tmux window by typing tmux 
3. Copy paste the commands, with a space at the bottom, into the tmux window. 
4. Detach from tmux by typing ctrl + b then d
5. Type tmux attach to re-attach to the tmux window.
6. If a second tmux window is opened, they can be toggled between ctrl + b then s. 

### Downloading the fasta files of contigs and scaffolds

On the local machine, make corresponding directories and scp only the fasta files. Here is an example:  
  
mkdir ELS-10x  
scp -i /Users/cameronstrachan/Desktop/scripts/MyFirstKey.pem ubuntu@52.33.23.163:/home/ubuntu/data/output/ELS-10x/*.fasta /Users/cameronstrachan/Desktop/ELS-10x  